In [1]:
import pandas as pd

In [2]:
gnd_data = pd.read_csv('../data/full_gnd.csv')

In [3]:
gnd_data.head()

,Unnamed: 0,id_str,created_at,full_text,retweet_count,favorite_count,user_id,screen_name,name,location
0,0,1029050376845176833,Mon Aug 13 17:01:43 +0000 2018,this kind of heat hasn't been normal since bef...,1,1,3955451301,OrinjNanu,Orinj Nanu 🌻,"Ohio, USA"
1,1,1029106823352250368,Mon Aug 13 20:46:01 +0000 2018,zinke on california fires: 'this is not a deba...,34,33,824797212425191425,ALT_uscis,ALT-immigration 🛂,NaN
2,2,1029110864534798336,Mon Aug 13 21:02:04 +0000 2018,is this a joke @dnc ? this is why we don't tru...,0,0,790774440284065792,whereisbastion,Bastion,"New York, New York"
3,3,1029116099076481031,Mon Aug 13 21:22:52 +0000 2018,precisely! #planetoverprofit #greennewdeal #cl...,0,0,49860693,bgaymoreoften,Brian Gay 🌻,United States
4,4,1029138574526935040,Mon Aug 13 22:52:11 +0000 2018,it's crucial to remember that dem centrism is ...,0,3,2377540724,SethLSanders,Carcass is just downtuned van Halen,UC Davis


In [4]:
filepath = '../NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], sep='\t', keep_default_na=False)
emolex_df.loc[emolex_df.word == 'green', 'association'] = 0
emolex_df.loc[emolex_df.word == 'deal', 'association'] = 0
emolex_df.head()

,word,emotion,association
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0


In [5]:
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
emolex_words.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,0,0,1,0,1,0,1,0,0
3,abandoned,1,0,0,1,0,1,0,1,0,0
4,abandonment,1,0,0,1,0,1,0,1,1,0


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
matrix = vec.fit_transform(gnd_data['full_text'])
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(vocabulary=emolex_words.word,
                      use_idf=False, 
                      norm='l1')
matrix = vec.fit_transform(gnd_data['full_text'])
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)
wordcount_df.head()

,aback,abacus,abandon,abandoned,abandonment,abate,abatement,abba,abbot,abbreviate,...,zephyr,zeppelin,zest,zip,zodiac,zone,zoo,zoological,zoology,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
negative_words = emolex_words[emolex_words.negative == 1].word
gnd_data['negative'] = wordcount_df[negative_words].sum(axis=1)
gnd_data.head(3)

positive_words = emolex_words[emolex_words.positive == 1].word
gnd_data['positive'] = wordcount_df[positive_words].sum(axis=1)
gnd_data.head(3)

,Unnamed: 0,id_str,created_at,full_text,retweet_count,favorite_count,user_id,screen_name,name,location,negative,positive
0,0,1029050376845176833,Mon Aug 13 17:01:43 +0000 2018,this kind of heat hasn't been normal since bef...,1,1,3955451301,OrinjNanu,Orinj Nanu 🌻,"Ohio, USA",0.076923,0.384615
1,1,1029106823352250368,Mon Aug 13 20:46:01 +0000 2018,zinke on california fires: 'this is not a deba...,34,33,824797212425191425,ALT_uscis,ALT-immigration 🛂,NaN,0.000000,0.333333
2,2,1029110864534798336,Mon Aug 13 21:02:04 +0000 2018,is this a joke @dnc ? this is why we don't tru...,0,0,790774440284065792,whereisbastion,Bastion,"New York, New York",0.300000,0.100000


In [9]:
# anger, anticipation, disgust, fear, joy, sadness, surprise, trust

anger_words = emolex_words[emolex_words.anger == 1].word
gnd_data['anger'] = wordcount_df[anger_words].sum(axis=1)
gnd_data.head(3)

anticipation_words = emolex_words[emolex_words.anticipation == 1].word
gnd_data['anticipation'] = wordcount_df[anticipation_words].sum(axis=1)
gnd_data.head(3)

disgust_words = emolex_words[emolex_words.disgust == 1].word
gnd_data['disgust'] = wordcount_df[disgust_words].sum(axis=1)
gnd_data.head(3)

fear_words = emolex_words[emolex_words.fear == 1].word
gnd_data['fear'] = wordcount_df[fear_words].sum(axis=1)
gnd_data.head(3)

joy_words = emolex_words[emolex_words.joy == 1].word
gnd_data['joy'] = wordcount_df[joy_words].sum(axis=1)
gnd_data.head(3)

sadness_words = emolex_words[emolex_words.sadness == 1].word
gnd_data['sadness'] = wordcount_df[sadness_words].sum(axis=1)
gnd_data.head(3)

surprise_words = emolex_words[emolex_words.surprise == 1].word
gnd_data['surprise'] = wordcount_df[surprise_words].sum(axis=1)
gnd_data.head(3)

trust_words = emolex_words[emolex_words.trust == 1].word
gnd_data['trust'] = wordcount_df[trust_words].sum(axis=1)
gnd_data.head(3)

,Unnamed: 0,id_str,created_at,full_text,retweet_count,favorite_count,user_id,screen_name,name,location,negative,positive,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,0,1029050376845176833,Mon Aug 13 17:01:43 +0000 2018,this kind of heat hasn't been normal since bef...,1,1,3955451301,OrinjNanu,Orinj Nanu 🌻,"Ohio, USA",0.076923,0.384615,0.076923,0.076923,0.0,0.076923,0.153846,0.0,0.0,0.384615
1,1,1029106823352250368,Mon Aug 13 20:46:01 +0000 2018,zinke on california fires: 'this is not a deba...,34,33,824797212425191425,ALT_uscis,ALT-immigration 🛂,NaN,0.000000,0.333333,0.000000,0.000000,0.0,0.333333,0.000000,0.0,0.0,0.000000
2,2,1029110864534798336,Mon Aug 13 21:02:04 +0000 2018,is this a joke @dnc ? this is why we don't tru...,0,0,790774440284065792,whereisbastion,Bastion,"New York, New York",0.300000,0.100000,0.100000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.300000


In [10]:
gnd_data.drop(columns={"Unnamed: 0"}, inplace=True)
gnd_data

,id_str,created_at,full_text,retweet_count,favorite_count,user_id,screen_name,name,location,negative,positive,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,1029050376845176833,Mon Aug 13 17:01:43 +0000 2018,this kind of heat hasn't been normal since bef...,1,1,3955451301,OrinjNanu,Orinj Nanu 🌻,"Ohio, USA",0.076923,0.384615,0.076923,0.076923,0.0000,0.076923,0.153846,0.0,0.000000,0.384615
1,1029106823352250368,Mon Aug 13 20:46:01 +0000 2018,zinke on california fires: 'this is not a deba...,34,33,824797212425191425,ALT_uscis,ALT-immigration 🛂,NaN,0.000000,0.333333,0.000000,0.000000,0.0000,0.333333,0.000000,0.0,0.000000,0.000000
2,1029110864534798336,Mon Aug 13 21:02:04 +0000 2018,is this a joke @dnc ? this is why we don't tru...,0,0,790774440284065792,whereisbastion,Bastion,"New York, New York",0.300000,0.100000,0.100000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.300000
3,1029116099076481031,Mon Aug 13 21:22:52 +0000 2018,precisely! #planetoverprofit #greennewdeal #cl...,0,0,49860693,bgaymoreoften,Brian Gay 🌻,United States,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000
4,1029138574526935040,Mon Aug 13 22:52:11 +0000 2018,it's crucial to remember that dem centrism is ...,0,3,2377540724,SethLSanders,Carcass is just downtuned van Halen,UC Davis,0.090909,0.363636,0.000000,0.000000,0.0000,0.000000,0.090909,0.0,0.000000,0.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27283,950940873461780480,2018-01-10 04:02:26+00:00,Thankfully @FERC rejected Trump's plan to bail...,1,1,2189460066,PghGreenLeft,Pittsburgh Green Left 🌻🥑,"Pittsburgh, PA",0.187500,0.187500,0.062500,0.125000,0.0625,0.125000,0.187500,0.0,0.125000,0.125000
27284,950953464242343936,2018-01-10 04:52:28+00:00,#ClimateBall Slap Shot https://t.co/pgnDwSWvit,0,1,228166929,nevaudit,Willard 👻,neverendingaudit.tumblr.com,1.000000,0.000000,1.000000,0.000000,0.0000,0.500000,0.000000,0.5,1.000000,0.000000
27285,950985996316938240,2018-01-10 07:01:44+00:00,Germany is at even higher latitudes (less sun)...,1,3,857062476961251328,AlleghenyGreens,Green Party of Allegheny County 🌻,"Pittsburgh, PA",0.000000,0.333333,0.000000,0.333333,0.0000,0.000000,0.333333,0.0,0.333333,0.333333
27286,951164295307563008,2018-01-10 18:50:14+00:00,New York sues big oil companies over climate c...,46,77,18949452,FT,Financial Times,London,0.000000,0.000000,0.000000,0.000000,0.0000,0.250000,0.000000,0.0,0.000000,0.000000


In [11]:
gnd_data.to_csv('../data/full_gnd_emolex_modified.csv', index=False)